In [ ]:
import random
from sage.graphs.graph_generators import graphs

def generate_outerplanar_graphs(n, k, prob_choose_from_list, prob_add_edge, prob_join_graph, prob_connect_graph):
    outerplanar_graphs = []  # Seznam za shranjevanje generiranih grafov.
    i = 1  # Števec za izpis števila ponovitev.

    for _ in range(k):
        print(f"Iteration {i}")  # Izpis trenutne iteracije.
        i += 1
        
        if outerplanar_graphs and random.random() < prob_choose_from_list: # Naključno odločanje, ali bomo vzeli graf iz seznama ali ustvarili nov cikel in ga dodali v seznam.
            chosen_graph = random.choice(outerplanar_graphs).copy() # Izbere graf naključno iz seznama.
            operation_choice = random.random() # Naključno odločanje, ali bomo dodali povezavo, združili dva grafa ali ju povezali z eno povezavo.

            if operation_choice < prob_add_edge:
                # Doda povezavo med dvema nepovezanima vozliščema, brez sekanja povezav.
                vertices = list(chosen_graph.vertices()) 
                added_edge = False
                for _ in range(len(vertices)**2):  # Omejitev števila poskusov.
                    u, v = random.sample(vertices, 2)
                    if not chosen_graph.has_edge(u, v):
                        new_graph = chosen_graph.copy()
                        new_graph.add_edge(u, v)
                        if new_graph.is_circular_planar():  # Preveri, ali je dobljen graf zunanjeravninski.
                            chosen_graph = new_graph
                            added_edge = True
                            break
                if not added_edge:
                    continue  # Če ni dodana nova povezava, nadaljuje z naslednjo iteracijo.

            elif operation_choice < prob_add_edge + prob_join_graph:
                # Združi dva grafa tako, da ju združi s skupnim vozliščem.
                other_graph = random.choice(outerplanar_graphs).copy()  # Izbere drug graf za združitev.
                if chosen_graph == other_graph:
                    continue  # Preskoči, če sta izbrana grafa ista.
                    
                u = random.choice(list(chosen_graph.vertices())) # Izbere naključni vozlišč iz obeh grafov.
                v = random.choice(list(other_graph.vertices()))

                new_graph = chosen_graph.disjoint_union(other_graph)  # Združi dva grafa tako, da nista povezana.
                new_graph.relabel() # Graf na novo označi.

                v_adjusted = v + len(chosen_graph.vertices()) # Prilagodi indeks za vozlišče v drugem grafu po združitvi.

                merged_vertex = max(new_graph.vertices()) + 1  # Doda povezave za novo vozlišče.
                new_graph.add_vertex(merged_vertex)

                u_neighbors = list(chosen_graph.neighbors(u))
                v_neighbors = list(other_graph.neighbors(v))

                for neighbor in u_neighbors:
                    new_graph.add_edge(merged_vertex, neighbor)

                for neighbor in v_neighbors:
                    adjusted_neighbor = neighbor + len(chosen_graph.vertices()) # Prilagodi sosedovo vozlišče glede na združeni graf.
                    if merged_vertex != adjusted_neighbor:  # Izogiba se samopovezavam.
                        new_graph.add_edge(merged_vertex, adjusted_neighbor)

                if u in new_graph: # Odstrani izvorna vozlišča "u" in "v" in njune povezave.
                    new_graph.delete_vertex(u)

                if v_adjusted in new_graph:
                    new_graph.delete_vertex(v_adjusted)

                if new_graph.is_circular_planar():  # Preveri, ali graf ostaja zunanjeravninski.
                    chosen_graph = new_graph

            elif operation_choice < prob_add_edge + prob_join_graph + prob_connect_graph: # Poveže dva grafa z eno naključno povezavo.
                other_graph = random.choice(outerplanar_graphs).copy()  # Izbere drug graf iz seznama.
                if chosen_graph == other_graph:
                    continue  # Preskoči, če sta izbrana grafa ista.

                new_graph = chosen_graph.disjoint_union(other_graph)  # Združi oba grafa v disjunktno unijo.
                new_graph.relabel()  # Graf preimenuje vozlišča, da se ohrani enoličnost.

                u = random.choice(list(chosen_graph.vertices()))  # Izbere naključno vozlišče iz prvega grafa.
                v = random.choice(list(other_graph.vertices()))  # Izbere naključno vozlišče iz drugega grafa.
                v_adjusted = v + len(chosen_graph.vertices())  # Prilagodi indeks vozlišča iz drugega grafa.

                new_graph.add_edge(u, v_adjusted)  # Doda povezavo med naključnima vozliščema.

                if new_graph.is_circular_planar():  # Preveri, ali graf ostaja zunanjeravninski.
                    chosen_graph = new_graph

            outerplanar_graphs.append(chosen_graph) # Doda dobljeni graf v seznam.

        else:
            cycle_size = random.randint(3, n)  # Ustvari nov cikel in ga doda v seznam.
            new_cycle = graphs.CycleGraph(cycle_size)
            outerplanar_graphs.append(new_cycle)

    return outerplanar_graphs
